In [2]:
!pip install rouge datasets transformers openai torch meteor textstat numpy nltk bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Usi

In [3]:
!git clone https://github.com/google-research/bleurt.git

Cloning into 'bleurt'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 134 (delta 0), reused 17 (delta 0), pack-reused 116
Receiving objects: 100% (134/134), 31.28 MiB | 26.32 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [4]:
!curl -L https://storage.googleapis.com/bleurt-ckpts/bleurt-base-512.zip -o bleurt-base-512.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   133  100   133    0     0    775      0 --:--:-- --:--:-- --:--:--   777


In [5]:
%cd /content/bleurt

/content/bleurt


In [6]:
!python setup.py install

running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based t

##GPT3.5 Turbo

GPT3.5 Turbo 1/3

In [20]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import requests
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

API_KEY = "sk-proj-bdyutzySqxWx1T89BKfzT3BlbkFJ53hs6AESe8RID5yR5m3q"
API_URL = "https://api.openai.com/v1/chat/completions"
headers = { "Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}" }

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score_val = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score_val

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def generate_and_evaluate_paragraph(prompt, reference):
    payload = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "You are a knowledgeable assistant for generating informative paragraphs."},
            {"role": "user", "content": f"{prompt}"}
        ],
        "max_tokens": 200,
        "n": 1,
        "stop": None,
        "temperature": 0.7,
    }
    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        generated_paragraph = response.json()["choices"][0]["message"]["content"].strip()
        # Calculate evaluation metrics
        meteor_score_val = calculate_meteor(generated_paragraph, reference)
        bertscore_val = calculate_bertscore(generated_paragraph, reference)
        readability_score = calculate_readability(generated_paragraph)
        bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
        # Printing the metrics
        print(f"Prompt: {prompt}")
        print(f"Generated Paragraph: {generated_paragraph}")
        print(f"METEOR Score: {meteor_score_val}")
        print(f"BERTScore: {bertscore_val}")
        print(f"Readability Score: {readability_score}")
        print(f"BLEURT Score: {bleurt_score_val}")
    except Exception as e:
        print("Error generating and evaluating paragraph:", e)

# User prompt and reference text
prompt = "What is cloud computing?"
# https://aws.amazon.com/what-is-cloud-computing/
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."
# Main function
generate_and_evaluate_paragraph(prompt, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Prompt: What is cloud computing?
Generated Paragraph: Cloud computing refers to the delivery of computing services, including servers, storage, databases, networking, software, and analytics, over the internet ("the cloud"). This technology allows individuals and businesses to access and use resources on a pay-as-you-go basis, without the need for large upfront investments in infrastructure. Cloud computing offers flexibility, scalability, and cost-efficiency, enabling users to easily scale their resources up or down based on their needs. It also provides the ability to access data and applications from anywhere with an internet connection, making collaboration and remote work more seamless. Major cloud providers include Amazon Web Services (AWS), Microsoft Azure, and Google Cloud Platform.
METEOR Score: 0.4250004492605173
BERTScore: tensor([0.6939])
Readability Score: 13.8
BLEURT Score: 0.6616824865341187


GPT3.5 Turbo 2/3

In [14]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import requests
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

API_KEY = "sk-proj-bdyutzySqxWx1T89BKfzT3BlbkFJ53hs6AESe8RID5yR5m3q"
API_URL = "https://api.openai.com/v1/chat/completions"
headers = { "Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}" }

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score_val = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score_val

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def generate_and_evaluate_paragraph(prompt, reference):
    payload = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "You are a knowledgeable assistant for generating informative paragraphs."},
            {"role": "user", "content": f"{prompt}"}
        ],
        "max_tokens": 200,
        "n": 1,
        "stop": None,
        "temperature": 0.7,
    }
    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        generated_paragraph = response.json()["choices"][0]["message"]["content"].strip()
        # Calculate evaluation metrics
        meteor_score_val = calculate_meteor(generated_paragraph, reference)
        bertscore_val = calculate_bertscore(generated_paragraph, reference)
        readability_score = calculate_readability(generated_paragraph)
        bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
        # Printing the metrics
        print(f"Prompt: {prompt}")
        print(f"Generated Paragraph: {generated_paragraph}")
        print(f"METEOR Score: {meteor_score_val}")
        print(f"BERTScore: {bertscore_val}")
        print(f"Readability Score: {readability_score}")
        print(f"BLEURT Score: {bleurt_score_val}")
    except Exception as e:
        print("Error generating and evaluating paragraph:", e)

# User prompt and reference text
prompt = "What is cloud computing?"
# https://aws.amazon.com/what-is-cloud-computing/
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."
# Main function
generate_and_evaluate_paragraph(prompt, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Prompt: What is cloud computing?
Generated Paragraph: Cloud computing is a technology that allows the delivery of computing services, such as servers, storage, databases, networking, software, and analytics, over the internet. Instead of owning and maintaining physical servers or infrastructure, users can access these resources on a pay-as-you-go basis from a cloud service provider. This enables businesses and individuals to scale their computing resources up or down based on their needs, without the need for large upfront investments in hardware. Cloud computing provides flexibility, cost-efficiency, and scalability, making it a popular choice for businesses of all sizes.
METEOR Score: 0.47289088449770106
BERTScore: tensor([0.7187])
Readability Score: 14.4
BLEURT Score: 0.7057788968086243


GPT3.5 Turbo 3/3

In [22]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import requests
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

API_KEY = "sk-proj-bdyutzySqxWx1T89BKfzT3BlbkFJ53hs6AESe8RID5yR5m3q"
API_URL = "https://api.openai.com/v1/chat/completions"
headers = { "Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}" }

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score_val = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score_val

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def generate_and_evaluate_paragraph(prompt, reference):
    payload = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "You are a knowledgeable assistant for generating informative paragraphs."},
            {"role": "user", "content": f"{prompt}"}
        ],
        "max_tokens": 200,
        "n": 1,
        "stop": None,
        "temperature": 0.7,
    }
    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        generated_paragraph = response.json()["choices"][0]["message"]["content"].strip()
        # Calculate evaluation metrics
        meteor_score_val = calculate_meteor(generated_paragraph, reference)
        bertscore_val = calculate_bertscore(generated_paragraph, reference)
        readability_score = calculate_readability(generated_paragraph)
        bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
        # Printing the metrics
        print(f"Prompt: {prompt}")
        print(f"Generated Paragraph: {generated_paragraph}")
        print(f"METEOR Score: {meteor_score_val}")
        print(f"BERTScore: {bertscore_val}")
        print(f"Readability Score: {readability_score}")
        print(f"BLEURT Score: {bleurt_score_val}")
    except Exception as e:
        print("Error generating and evaluating paragraph:", e)

# User prompt and reference text
prompt = "What is cloud computing?"
# https://aws.amazon.com/what-is-cloud-computing/
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."
# Main function
generate_and_evaluate_paragraph(prompt, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Prompt: What is cloud computing?
Generated Paragraph: Cloud computing refers to the delivery of computing services, including storage, processing power, and software applications, over the internet. Instead of storing data and running applications on a local server or personal computer, cloud computing allows users to access these resources remotely through a network of servers hosted on the internet. This enables individuals and businesses to scale their computing needs on-demand, pay only for the resources they use, and access their data and applications from any device with an internet connection. Cloud computing offers benefits such as cost-efficiency, scalability, flexibility, and improved collaboration and accessibility. Major cloud service providers include Amazon Web Services (AWS), Microsoft Azure, and Google Cloud Platform.
METEOR Score: 0.44879544555325984
BERTScore: tensor([0.6902])
Readability Score: 15.5
BLEURT Score: 0.6454196572303772


## GPT4 Turbo

GPT4 1/3

In [17]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import requests
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

API_KEY = "sk-proj-bdyutzySqxWx1T89BKfzT3BlbkFJ53hs6AESe8RID5yR5m3q"
API_URL = "https://api.openai.com/v1/chat/completions"
headers = { "Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}" }

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score_val = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score_val

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def generate_and_evaluate_paragraph(prompt, reference):
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {"role": "system", "content": "You are a knowledgeable assistant for generating informative paragraphs."},
            {"role": "user", "content": f"{prompt}"}
        ],
        "max_tokens": 200,
        "n": 1,
        "stop": None,
        "temperature": 0.7,
    }
    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        generated_paragraph = response.json()["choices"][0]["message"]["content"].strip()
        # Calculate evaluation metrics
        meteor_score_val = calculate_meteor(generated_paragraph, reference)
        bertscore_val = calculate_bertscore(generated_paragraph, reference)
        readability_score = calculate_readability(generated_paragraph)
        bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
        # Printing the metrics
        print(f"Prompt: {prompt}")
        print(f"Generated Paragraph: {generated_paragraph}")
        print(f"METEOR Score: {meteor_score_val}")
        print(f"BERTScore: {bertscore_val}")
        print(f"Readability Score: {readability_score}")
        print(f"BLEURT Score: {bleurt_score_val}")
    except Exception as e:
        print("Error generating and evaluating paragraph:", e)

# User prompt and reference text
prompt = "What is cloud computing?"
# https://aws.amazon.com/what-is-cloud-computing/
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."
# Main function
generate_and_evaluate_paragraph(prompt, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Prompt: What is cloud computing?
Generated Paragraph: Cloud computing is a technology that allows individuals and organizations to access and utilize computing resources such as servers, storage, databases, networking, software, and more, over the internet. This model provides the flexibility to scale up or down resources on-demand, thereby offering cost efficiency and the ability to handle varying workloads without the need to invest in physical hardware and infrastructure. The services offered in cloud computing are generally divided into three primary categories: Infrastructure as a Service (IaaS), Platform as a Service (PaaS), and Software as a Service (SaaS). These services enable users to run applications and store data on remote servers, access software and platforms through web browsers, and leverage powerful computing resources without the direct management of underlying hardware. Major providers of cloud computing services include Amazon Web Services (AWS), Microsoft Azure, a

GPT4 2/3

In [18]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import requests
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

API_KEY = "sk-proj-bdyutzySqxWx1T89BKfzT3BlbkFJ53hs6AESe8RID5yR5m3q"
API_URL = "https://api.openai.com/v1/chat/completions"
headers = { "Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}" }

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score_val = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score_val

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def generate_and_evaluate_paragraph(prompt, reference):
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {"role": "system", "content": "You are a knowledgeable assistant for generating informative paragraphs."},
            {"role": "user", "content": f"{prompt}"}
        ],
        "max_tokens": 200,
        "n": 1,
        "stop": None,
        "temperature": 0.7,
    }
    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        generated_paragraph = response.json()["choices"][0]["message"]["content"].strip()
        # Calculate evaluation metrics
        meteor_score_val = calculate_meteor(generated_paragraph, reference)
        bertscore_val = calculate_bertscore(generated_paragraph, reference)
        readability_score = calculate_readability(generated_paragraph)
        bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
        # Printing the metrics
        print(f"Prompt: {prompt}")
        print(f"Generated Paragraph: {generated_paragraph}")
        print(f"METEOR Score: {meteor_score_val}")
        print(f"BERTScore: {bertscore_val}")
        print(f"Readability Score: {readability_score}")
        print(f"BLEURT Score: {bleurt_score_val}")
    except Exception as e:
        print("Error generating and evaluating paragraph:", e)

# User prompt and reference text
prompt = "What is cloud computing?"
# https://aws.amazon.com/what-is-cloud-computing/
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."
# Main function
generate_and_evaluate_paragraph(prompt, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Prompt: What is cloud computing?
Generated Paragraph: Cloud computing is a technology that allows individuals and organizations to access and use computing resources such as servers, storage, databases, networking, software, and more over the internet. This means that users can utilize high-end computing capabilities without having to own and maintain physical infrastructure, facilitating cost savings, scalability, and flexibility. The resources can be provisioned and scaled up or down quickly to suit varying demand, making it an efficient solution for businesses of all sizes. Major types of cloud services include Infrastructure as a Service (IaaS), Platform as a Service (PaaS), and Software as a Service (SaaS), each providing different levels of control, management, and flexibility. Cloud computing has become foundational in the digital transformation strategies of many businesses, enabling innovations like big data analytics, artificial intelligence, and the Internet of Things (IoT).

GPT4 3/3

In [19]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import requests
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

API_KEY = "sk-proj-bdyutzySqxWx1T89BKfzT3BlbkFJ53hs6AESe8RID5yR5m3q"
API_URL = "https://api.openai.com/v1/chat/completions"
headers = { "Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}" }

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score_val = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score_val

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def generate_and_evaluate_paragraph(prompt, reference):
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {"role": "system", "content": "You are a knowledgeable assistant for generating informative paragraphs."},
            {"role": "user", "content": f"{prompt}"}
        ],
        "max_tokens": 200,
        "n": 1,
        "stop": None,
        "temperature": 0.7,
    }
    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        generated_paragraph = response.json()["choices"][0]["message"]["content"].strip()
        # Calculate evaluation metrics
        meteor_score_val = calculate_meteor(generated_paragraph, reference)
        bertscore_val = calculate_bertscore(generated_paragraph, reference)
        readability_score = calculate_readability(generated_paragraph)
        bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
        # Printing the metrics
        print(f"Prompt: {prompt}")
        print(f"Generated Paragraph: {generated_paragraph}")
        print(f"METEOR Score: {meteor_score_val}")
        print(f"BERTScore: {bertscore_val}")
        print(f"Readability Score: {readability_score}")
        print(f"BLEURT Score: {bleurt_score_val}")
    except Exception as e:
        print("Error generating and evaluating paragraph:", e)

# User prompt and reference text
prompt = "What is cloud computing?"
# https://aws.amazon.com/what-is-cloud-computing/
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."
# Main function
generate_and_evaluate_paragraph(prompt, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Prompt: What is cloud computing?
Generated Paragraph: Cloud computing is a technology that allows individuals and organizations to access and store data and applications on remote servers, which are then accessed via the internet. This means that users don't need to manage physical servers themselves or run software applications on their own machines. Cloud computing services can be divided into three main categories: Infrastructure as a Service (IaaS), Platform as a Service (PaaS), and Software as a Service (SaaS). IaaS provides basic storage and computing capabilities as online services, PaaS offers hardware and software tools available over the internet, typically for application development, and SaaS delivers software applications over the internet, on a subscription basis. Cloud computing enables flexibility, scalability, and cost efficiency, as it typically operates on a pay-as-you-go model, allowing users to scale services to fit their needs, customize applications, and access c

## Manually injecting responses from chatbots

## Google Gemini

Gemini 1/3

In [25]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Imagine having access to a vast pool of computer resources like storage and processing power, whenever you need it, over the internet. That's the basic idea behind cloud computing. Instead of managing your own physical servers, you rent virtual space on a cloud provider's infrastructure. This allows you to scale resources up or down on demand, pay only for what you use, and access your data from anywhere with an internet connection. Cloud computing is a game-changer for businesses of all sizes, offering increased flexibility, scalability, and cost-efficiency."

evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Imagine having access to a vast pool of computer resources like storage and processing power, whenever you need it, over the internet. That's the basic idea behind cloud computing. Instead of managing your own physical servers, you rent virtual space on a cloud provider's infrastructure. This allows you to scale resources up or down on demand, pay only for what you use, and access your data from anywhere with an internet connection. Cloud computing is a game-changer for businesses of all sizes, offering increased flexibility, scalability, and cost-efficiency.
METEOR Score: 0.3177004538577912
BERTScore: tensor([0.6181])
Readability Score: 11.3
BLEURT Score: 0.5736182332038879


Gemini 2/3

In [27]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Imagine renting computing power instead of buying your own physical hardware. Cloud computing is exactly that - it's the on-demand delivery of IT resources like storage, databases, servers, and software over the internet.  This cloud eliminates the need for companies and individuals to manage their own physical infrastructure, offering benefits like scalability, cost-efficiency, and increased agility.  In simpler terms, it's like paying for electricity instead of owning your own power plant."

evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Imagine renting computing power instead of buying your own physical hardware. Cloud computing is exactly that - it's the on-demand delivery of IT resources like storage, databases, servers, and software over the internet.  This cloud eliminates the need for companies and individuals to manage their own physical infrastructure, offering benefits like scalability, cost-efficiency, and increased agility.  In simpler terms, it's like paying for electricity instead of owning your own power plant.
METEOR Score: 0.38172209722702977
BERTScore: tensor([0.6346])
Readability Score: 13.7
BLEURT Score: 0.5528774857521057


Gemini 3/3

In [28]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Imagine having access to a vast pool of computer resources like storage space and processing power, whenever you need it, over the internet. That's essentially cloud computing. It's on-demand delivery of IT services, from data storage to software, that you access from a cloud provider instead of managing your own physical servers. This allows for scalability and flexibility, along with potential cost savings, as you only pay for what you use."
evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Imagine having access to a vast pool of computer resources like storage space and processing power, whenever you need it, over the internet. That's essentially cloud computing. It's on-demand delivery of IT services, from data storage to software, that you access from a cloud provider instead of managing your own physical servers. This allows for scalability and flexibility, along with potential cost savings, as you only pay for what you use.
METEOR Score: 0.4875728640091469
BERTScore: tensor([0.6391])
Readability Score: 10.2
BLEURT Score: 0.5992104411125183


## Gemma-7b-it




Gemma-7b-it 1/3

In [29]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Cloud computing is a revolutionary approach to computing that enables on-demand access to a shared pool of computing resources, such as servers, storage, databases, and software applications over a network. It offers users the ability to scale resources up or down as needed, eliminating the need for upfront infrastructure investment and management. Cloud computing models, such as Infrastructure as a Service (IaaS), Platform as a Service (PaaS), and Software as a Service (SaaS), provide users with flexibility, scalability, and cost-effectiveness. By leveraging the power of the cloud, businesses and individuals can access computing resources from anywhere, anytime, and scale their needs effortlessly to meet evolving requirements."
evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Cloud computing is a revolutionary approach to computing that enables on-demand access to a shared pool of computing resources, such as servers, storage, databases, and software applications over a network. It offers users the ability to scale resources up or down as needed, eliminating the need for upfront infrastructure investment and management. Cloud computing models, such as Infrastructure as a Service (IaaS), Platform as a Service (PaaS), and Software as a Service (SaaS), provide users with flexibility, scalability, and cost-effectiveness. By leveraging the power of the cloud, businesses and individuals can access computing resources from anywhere, anytime, and scale their needs effortlessly to meet evolving requirements.
METEOR Score: 0.36607884432842897
BERTScore: tensor([0.6304])
Readability Score: 16.0
BLEURT Score: 0.5691852569580078


Gemma-7b-it 2/3

In [30]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Cloud computing is a revolutionary approach to computing that enables on-demand access to a shared pool of computing resources over a network. It offers users the ability to access computing services such as storage, processing power, and software applications over the internet, rather than relying on physical hardware or infrastructure. Cloud computing models provide scalability, flexibility, and cost-effectiveness, allowing users to adjust their computing needs as required. It eliminates the need for organizations to invest heavily in hardware and infrastructure, reducing capital expenditures and IT management costs. By leveraging cloud computing, businesses and individuals can access the computing power they need, when they need it, from anywhere with an internet connection."
evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Cloud computing is a revolutionary approach to computing that enables on-demand access to a shared pool of computing resources over a network. It offers users the ability to access computing services such as storage, processing power, and software applications over the internet, rather than relying on physical hardware or infrastructure. Cloud computing models provide scalability, flexibility, and cost-effectiveness, allowing users to adjust their computing needs as required. It eliminates the need for organizations to invest heavily in hardware and infrastructure, reducing capital expenditures and IT management costs. By leveraging cloud computing, businesses and individuals can access the computing power they need, when they need it, from anywhere with an internet connection.
METEOR Score: 0.368219656283566
BERTScore: tensor([0.6406])
Readability Score: 15.5
BLEURT Score: 0.57914799451828


Gemma-7b-it 3/3

In [31]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Cloud computing is a revolutionary approach to computing that enables users to access on-demand computing resources, such as servers, storage, databases, and software applications, over a network rather than relying on physical hardware or local infrastructure. It offers a pay-as-you-go model, allowing users to scale resources up or down as needed. Cloud computing services are delivered through various models, including Infrastructure as a Service (IaaS), Platform as a Service (PaaS), and Software as a Service (SaaS). This decentralized and scalable approach to computing empowers individuals and organizations to access essential digital services from anywhere, anytime, while eliminating the need for costly hardware investments and infrastructure management."
evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Cloud computing is a revolutionary approach to computing that enables users to access on-demand computing resources, such as servers, storage, databases, and software applications, over a network rather than relying on physical hardware or local infrastructure. It offers a pay-as-you-go model, allowing users to scale resources up or down as needed. Cloud computing services are delivered through various models, including Infrastructure as a Service (IaaS), Platform as a Service (PaaS), and Software as a Service (SaaS). This decentralized and scalable approach to computing empowers individuals and organizations to access essential digital services from anywhere, anytime, while eliminating the need for costly hardware investments and infrastructure management.
METEOR Score: 0.28467153284671537
BERTScore: tensor([0.6591])
Readability Score: 17.2
BLEURT Score: 0.6128228306770325


## Llama3-8b-8192

Llama3-8b-8192 1/3

In [32]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Cloud computing is a model of delivering computing services over the internet, where resources such as servers, storage, databases, software, and applications are provided as a service to users on-demand. Instead of having to manage and maintain physical hardware and software, users can access these resources on a pay-as-you-go basis, allowing for greater flexibility, scalability, and cost savings. With cloud computing, users can access their data and applications from anywhere, at any time, and on any device with an internet connection. This has revolutionized the way businesses operate, enabling them to quickly scale up or down to meet changing demands, reduce capital and operational expenses, and improve collaboration and productivity."
evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Cloud computing is a model of delivering computing services over the internet, where resources such as servers, storage, databases, software, and applications are provided as a service to users on-demand. Instead of having to manage and maintain physical hardware and software, users can access these resources on a pay-as-you-go basis, allowing for greater flexibility, scalability, and cost savings. With cloud computing, users can access their data and applications from anywhere, at any time, and on any device with an internet connection. This has revolutionized the way businesses operate, enabling them to quickly scale up or down to meet changing demands, reduce capital and operational expenses, and improve collaboration and productivity.
METEOR Score: 0.41083856441048033
BERTScore: tensor([0.6766])
Readability Score: 16.4
BLEURT Score: 0.6611599922180176


Llama3-8b-8192 2/3

In [33]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Cloud computing is a model of delivering computing services over the internet, where resources such as servers, storage, databases, software, and applications are provided as a service to users on-demand. Instead of having to manage and maintain physical hardware and software, users can access these resources on a pay-as-you-go basis, allowing for greater flexibility, scalability, and cost savings. Cloud computing enables users to quickly scale up or down to meet changing business needs, and access a wide range of applications and services from anywhere, at any time. This shift from traditional on-premise computing to cloud computing has revolutionized the way businesses operate, allowing for increased agility, collaboration, and innovation."
evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Cloud computing is a model of delivering computing services over the internet, where resources such as servers, storage, databases, software, and applications are provided as a service to users on-demand. Instead of having to manage and maintain physical hardware and software, users can access these resources on a pay-as-you-go basis, allowing for greater flexibility, scalability, and cost savings. Cloud computing enables users to quickly scale up or down to meet changing business needs, and access a wide range of applications and services from anywhere, at any time. This shift from traditional on-premise computing to cloud computing has revolutionized the way businesses operate, allowing for increased agility, collaboration, and innovation.
METEOR Score: 0.3498819500765973
BERTScore: tensor([0.6722])
Readability Score: 16.3
BLEURT Score: 0.6707401871681213


Llama3-8b-8192 3/3

In [34]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Cloud computing is a model of delivering computing services over the internet, where resources such as servers, storage, databases, software, and applications are provided as a service to users on-demand. Instead of having to manage and maintain physical hardware and software, users can access these resources on a pay-as-you-go basis, allowing for greater flexibility, scalability, and cost savings. Cloud computing enables users to store and process large amounts of data, run applications, and collaborate with others in a virtual environment. This model has revolutionized the way businesses and individuals work, allowing for greater collaboration, increased productivity, and reduced costs. With cloud computing, users can access their data and applications from anywhere, at any time, and on any device with an internet connection."
evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Cloud computing is a model of delivering computing services over the internet, where resources such as servers, storage, databases, software, and applications are provided as a service to users on-demand. Instead of having to manage and maintain physical hardware and software, users can access these resources on a pay-as-you-go basis, allowing for greater flexibility, scalability, and cost savings. Cloud computing enables users to store and process large amounts of data, run applications, and collaborate with others in a virtual environment. This model has revolutionized the way businesses and individuals work, allowing for greater collaboration, increased productivity, and reduced costs. With cloud computing, users can access their data and applications from anywhere, at any time, and on any device with an internet connection.
METEOR Score: 0.4204257676479899
BERTScore: tensor([0.6721])
Readability Score: 15.2
BLEURT Score: 0.6932727098464966


## Llama3-70b-8192

Llama3-70b-8192 1/3

In [36]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Cloud computing is a revolutionary technology that enables on-demand access to a shared pool of computing resources, such as servers, storage, databases, software, and applications, over the internet. Instead of storing and processing data on a personal computer or local server, cloud computing allows users to store, manage, and process data remotely, using a network of remote servers accessed through the internet. This enables users to access their data and applications from anywhere, at any time, and on any device with an internet connection. Cloud computing provides a scalable, flexible, and cost-effective way to manage and process large amounts of data, making it an attractive option for businesses, organizations, and individuals alike. With cloud computing, users can easily scale up or down to meet changing business needs, reduce capital expenditures on hardware and software, and enjoy automatic software updates and maintenance, freeing up resources to focus on core business activities."
evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Cloud computing is a revolutionary technology that enables on-demand access to a shared pool of computing resources, such as servers, storage, databases, software, and applications, over the internet. Instead of storing and processing data on a personal computer or local server, cloud computing allows users to store, manage, and process data remotely, using a network of remote servers accessed through the internet. This enables users to access their data and applications from anywhere, at any time, and on any device with an internet connection. Cloud computing provides a scalable, flexible, and cost-effective way to manage and process large amounts of data, making it an attractive option for businesses, organizations, and individuals alike. With cloud computing, users can easily scale up or down to meet changing business needs, reduce capital expenditures on hardware and software, and enjoy automatic software updates and maintenance, freeing up resources to focus o

Llama3-70b-8192 2/3

In [37]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Cloud computing is a model of delivering computing services over the internet, where resources such as servers, storage, databases, software, and applications are provided as a service to users on-demand. Instead of having to manage and maintain physical hardware and software, users can access these resources on a pay-as-you-go basis, allowing for greater flexibility, scalability, and cost savings. Cloud computing enables users to store, manage, and process data remotely, using a network of remote servers accessed through the internet. This allows users to access their data and applications from anywhere, on any device, at any time, as long as they have an internet connection. Cloud computing also enables automatic software updates, reduced maintenance, and enhanced collaboration and productivity, making it an attractive option for individuals, businesses, and organizations of all sizes."
evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Cloud computing is a model of delivering computing services over the internet, where resources such as servers, storage, databases, software, and applications are provided as a service to users on-demand. Instead of having to manage and maintain physical hardware and software, users can access these resources on a pay-as-you-go basis, allowing for greater flexibility, scalability, and cost savings. Cloud computing enables users to store, manage, and process data remotely, using a network of remote servers accessed through the internet. This allows users to access their data and applications from anywhere, on any device, at any time, as long as they have an internet connection. Cloud computing also enables automatic software updates, reduced maintenance, and enhanced collaboration and productivity, making it an attractive option for individuals, businesses, and organizations of all sizes.
METEOR Score: 0.3834624228700227
BERTScore: tensor([0.6760])
Readability Score

Llama3-70b-8192 3/3

In [38]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Cloud computing is a revolutionary technology that enables on-demand access to a shared pool of computing resources, such as servers, storage, databases, software, and applications, over the internet. Instead of storing and processing data on personal computers or local servers, cloud computing allows users to store, manage, and process data remotely, using a network of remote servers accessed through the internet. This approach provides numerous benefits, including scalability, flexibility, and cost-effectiveness, as users only pay for the resources they use. Cloud computing also enables easy collaboration, automatic software updates, and enhanced data security, making it an attractive option for individuals, businesses, and organizations of all sizes. With cloud computing, users can access their data and applications from anywhere, at any time, using any device with an internet connection, making it an essential tool for today's digital age."
evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Cloud computing is a revolutionary technology that enables on-demand access to a shared pool of computing resources, such as servers, storage, databases, software, and applications, over the internet. Instead of storing and processing data on personal computers or local servers, cloud computing allows users to store, manage, and process data remotely, using a network of remote servers accessed through the internet. This approach provides numerous benefits, including scalability, flexibility, and cost-effectiveness, as users only pay for the resources they use. Cloud computing also enables easy collaboration, automatic software updates, and enhanced data security, making it an attractive option for individuals, businesses, and organizations of all sizes. With cloud computing, users can access their data and applications from anywhere, at any time, using any device with an internet connection, making it an essential tool for today's digital age.
METEOR Score: 0.33650

## Claude3

Claude3 1/3

In [24]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow_hub as hub
import bleurt
import numpy as np

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")
        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    scorer = bleurt.score.BleurtScorer(os.path.join(checkpoint_path, "BLEURT-20"))
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Cloud computing is a model for delivering computing resources, including storage, processing power, software, and services, over the internet on a pay-as-you-go basis. Instead of owning and maintaining physical data centers and servers, cloud computing allows individuals and organizations to access these resources virtually from a cloud provider's network of remote servers hosted in data centers. The cloud provider manages and maintains the underlying infrastructure, while users can provision and use the computing resources they need through a web-based interface or API. This approach offers scalability, flexibility, cost-efficiency, and eliminates the need for organizations to invest in expensive hardware and infrastructure maintenance. Cloud computing services can be broadly divided into three main categories: Infrastructure as a Service (IaaS), Platform as a Service (PaaS), and Software as a Service (SaaS), catering to different business needs and levels of abstraction."
evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Cloud computing is a model for delivering computing resources, including storage, processing power, software, and services, over the internet on a pay-as-you-go basis. Instead of owning and maintaining physical data centers and servers, cloud computing allows individuals and organizations to access these resources virtually from a cloud provider's network of remote servers hosted in data centers. The cloud provider manages and maintains the underlying infrastructure, while users can provision and use the computing resources they need through a web-based interface or API. This approach offers scalability, flexibility, cost-efficiency, and eliminates the need for organizations to invest in expensive hardware and infrastructure maintenance. Cloud computing services can be broadly divided into three main categories: Infrastructure as a Service (IaaS), Platform as a Service (PaaS), and Software as a Service (SaaS), catering to different business needs and levels of abst

Claude 2/3

In [11]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow as tf
import numpy as np

# Make sure to import bleurt correctly
from bleurt import score as bleurt_score

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score_val = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score_val

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    bleurt_model_path = os.path.join(checkpoint_path, "BLEURT-20")

    if not os.path.exists(bleurt_model_path):
        os.makedirs(checkpoint_path, exist_ok=True)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")

        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    # Initialize BLEURT scorer
    scorer = bleurt_score.BleurtScorer(bleurt_model_path)
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Cloud computing is the delivery of computing services - including servers, storage, databases, networking, software, analytics, and intelligence - over the Internet (the cloud) to offer faster innovation, flexible resources, and economies of scale. Cloud providers such as Amazon Web Services, Microsoft Azure, and Google Cloud Platform maintain and operate the network-accessible computing infrastructure required for these services. Instead of owning their own computing infrastructure or data centers, companies can rent access to cloud services from a cloud provider and consume those services as needed over the internet. This cloud computing model allows businesses to reduce upfront capital costs, gain flexibility and scalability, and avoid the overhead and expertise required for operating on-premises infrastructure."
evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Cloud computing is the delivery of computing services - including servers, storage, databases, networking, software, analytics, and intelligence - over the Internet (the cloud) to offer faster innovation, flexible resources, and economies of scale. Cloud providers such as Amazon Web Services, Microsoft Azure, and Google Cloud Platform maintain and operate the network-accessible computing infrastructure required for these services. Instead of owning their own computing infrastructure or data centers, companies can rent access to cloud services from a cloud provider and consume those services as needed over the internet. This cloud computing model allows businesses to reduce upfront capital costs, gain flexibility and scalability, and avoid the overhead and expertise required for operating on-premises infrastructure.
METEOR Score: 0.4197373067290174
BERTScore: tensor([0.6714])
Readability Score: 17.8
BLEURT Score: 0.6594517230987549


Claude 3/3

In [13]:
import os
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import textstat
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from bert_score import score
import warnings
import tensorflow as tf
import numpy as np

# Make sure to import bleurt correctly
from bleurt import score as bleurt_score

warnings.filterwarnings("ignore", message="Some weights of RobertaModel were not initialized from the model checkpoint")

# Function to calculate METEOR score
def calculate_meteor(hypothesis, reference):
    hypothesis_tokens = word_tokenize(hypothesis.lower())
    reference_tokens = word_tokenize(reference.lower())
    return meteor_score([reference_tokens], hypothesis_tokens)

# Function to calculate BERTScore
def calculate_bertscore(hypothesis, reference, model_type="bert-base-uncased"):
    _, _, bert_score_val = score([hypothesis], [reference], lang="en", model_type=model_type)
    return bert_score_val

# Function to calculate readability metrics
def calculate_readability(text):
    flesch_kincaid_grade = textstat.flesch_kincaid_grade(text)
    return flesch_kincaid_grade

# Function to calculate BLEURT score
def calculate_bleurt(hypothesis, reference):
    """Calculates BLEURT score."""
    checkpoint_path = "bleurt/test_checkpoint"
    bleurt_model_path = os.path.join(checkpoint_path, "BLEURT-20")

    if not os.path.exists(bleurt_model_path):
        os.makedirs(checkpoint_path, exist_ok=True)
        url = "https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip"
        zip_path = os.path.join(checkpoint_path, "BLEURT-20.zip")

        os.system(f"curl -L {url} -o {zip_path}")
        os.system(f"unzip {zip_path} -d {checkpoint_path}")

    # Initialize BLEURT scorer
    scorer = bleurt_score.BleurtScorer(bleurt_model_path)
    bleurt_score_val = scorer.score(references=[reference], candidates=[hypothesis])[0]
    return bleurt_score_val

def evaluate_paragraph(chatbot_response, reference):
    generated_paragraph = chatbot_response.strip()
    # Calculate evaluation metrics
    meteor_score_val = calculate_meteor(generated_paragraph, reference)
    bertscore_val = calculate_bertscore(generated_paragraph, reference)
    readability_score = calculate_readability(generated_paragraph)
    bleurt_score_val = calculate_bleurt(generated_paragraph, reference)
    # Printing the metrics
    print(f"Generated Paragraph: {generated_paragraph}")
    print(f"METEOR Score: {meteor_score_val}")
    print(f"BERTScore: {bertscore_val}")
    print(f"Readability Score: {readability_score}")
    print(f"BLEURT Score: {bleurt_score_val}")

prompt = "What is cloud computing?"
reference = "Cloud computing is the on-demand delivery of IT resources over the Internet with pay-as-you-go pricing. Instead of buying, owning, and maintaining physical data centers and servers, you can access technology services, such as computing power, storage, and databases, on an as-needed basis from a cloud provider like Amazon Web Services (AWS)."

# Hard coding chatbot response
chatbot_response = "Cloud computing refers to the delivery of computing services - including servers, storage, databases, networking, software, analytics, and intelligence - over the Internet (the cloud). Cloud services allow individuals and organizations to access and use these computing resources on an as-needed basis without having to purchase, install, and maintain the underlying hardware and software themselves. These resources are housed and maintained by cloud providers in their secure data center facilities. Cloud computing enables ubiquitous access to a shared pool of configurable computing resources that can be rapidly provisioned and released with minimal management effort or service provider interaction. It provides a flexible, scalable, and cost-effective model for delivering enterprise-grade IT capabilities to users and organizations worldwide."
evaluate_paragraph(chatbot_response, reference)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generated Paragraph: Cloud computing refers to the delivery of computing services - including servers, storage, databases, networking, software, analytics, and intelligence - over the Internet (the cloud). Cloud services allow individuals and organizations to access and use these computing resources on an as-needed basis without having to purchase, install, and maintain the underlying hardware and software themselves. These resources are housed and maintained by cloud providers in their secure data center facilities. Cloud computing enables ubiquitous access to a shared pool of configurable computing resources that can be rapidly provisioned and released with minimal management effort or service provider interaction. It provides a flexible, scalable, and cost-effective model for delivering enterprise-grade IT capabilities to users and organizations worldwide.
METEOR Score: 0.43001909508971686
BERTScore: tensor([0.6691])
Readability Score: 15.7
BLEURT Score: 0.6385501623153687
